In [2]:
from transformers import AutoModel,AutoTokenizer

device = 'cpu'
model = AutoModel.from_pretrained('google/mobilebert-uncased')
model = model.to(device)
model

c:\Users\Majid\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MobileBertModel(
  (embeddings): MobileBertEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
    (LayerNorm): NoNorm()
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): MobileBertEncoder(
    (layer): ModuleList(
      (0-23): 24 x MobileBertLayer(
        (attention): MobileBertAttention(
          (self): MobileBertSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): MobileBertSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (LayerNorm): NoNorm()
          )
      

In [4]:
tokenizer = AutoTokenizer.from_pretrained('google/mobilebert-uncased')
tokenizer

MobileBertTokenizerFast(name_or_path='google/mobilebert-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
import datasets
from torch.utils.data.dataloader import DataLoader

data =datasets.load_dataset('glue','mrpc')
train_encoded = data['train'].map(lambda x: tokenizer(x['sentence1'],x['sentence2'],padding='max_length',max_length=512))
print(train_encoded)
train_encoded.set_format(type='torch',columns=['label', 'input_ids', 'token_type_ids', 'attention_mask'])
train_loader = DataLoader(train_encoded,32)

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})


In [6]:
import torch
def step(x,y):
    out = model(x)
    print(out)

b = next(iter(train_loader))
x={}
y={}
for k in b.keys():
    if k=='label':
        y = b[k]
    else:
        x[k]=b[k].squeeze(dim=1).to(device)
    
model(**x)

: 